<a href="https://colab.research.google.com/github/tomonari-masada/course2025-sml/blob/main/04_nearest_neighbors_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# k近傍法 (k-nearest neighbors)
* 直感的には、「どんな友人と付き合っているかを見ればその人が分かる」という考え方にもとづく手法。
  * 近いものは同類、という考え方。
* 分類にも回帰にも使える。
  * 分類＝未知のデータ点について、それが属するクラスを予測
  * 回帰＝未知のデータ点について、対応する特定の数値を予測
* 新しいデータ点について、対応するターゲット（クラス or 数値）を予測する場合・・・
* k近傍法では、そのデータ点のk個の近傍のターゲットを元に予測する。
  * クラスを予測する（分類）の場合、予測は多数決で行われる。
  * 数値を予測する（回帰）の場合、近傍のターゲットの値の平均を予測値とする。（平均値以外もありうる。）

**注：今回と次回は、機械学習の授業でもあるのですが、ほぼほぼ、Pythonのコーディングの練習です。**

## 準備

### インポート

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'

### データファイル


* `lifesat_extended.csv`というファイルをColabにアップロードする。

In [ ]:
from google.colab import files
files.upload()

* 国名をインデックスにしてデータフレームを作る。

In [ ]:
df = pd.read_csv("lifesat_extended.csv", index_col="Country")
df

## 今回の問題設定
* この授業では、以下の二つを、予測のための特徴量(feature)として使うことにする。
  * 国民一人当たりのGDP (GDP per capita)
  * 雇用率 (employment rate)
* 予測するターゲットは、生活満足度 (life satisfaction) とする。
* そして、最終的に予測したいのは、日本の生活満足度だとする。
  * もちろん、すでに答えは分かっているが、最後まで正解は伏せておく。


* 言い換えれば、日本のデータが、テストデータとなる。
  * テストデータ＝最終的に手法の性能を評価するために使うデータ集合。
* テストデータでの評価結果を見て、手法をチューニングしてはいけないです！
  * 機械学習の基本中の基本。
* そこで、日本を除外したデータフレームを作り、それを使う。
  * 日本のデータは、最後の最後に、手法評価時に使うだけ。

* （ちょっとコメント）
  * 普通は、テストデータは今回のように一つだけではなく、複数のインスタンスを用意する。
  * 今回は、全体のデータ数が29個と少ないため、テストデータは一つだけにしている。

In [ ]:
df_train = df.drop(['Japan'])
df_test = df.loc[['Japan']]

* 訓練データのさまざまな情報を確認する。

In [ ]:
df_train

In [ ]:
df_train.describe()

In [ ]:
df_train.info()

* テストデータは、最後の最後に、チューニング済みの手法を評価する時まで、使わない。

* GDP per capitaやemployment rateを使って生活満足度を予測するという問題設定が、そもそも妥当な問題設定かどうか、確認する。
 * 特徴量とターゲットとのあいだに、ある程度の関連性がないと、解けない問題を解こうとしていることになる。

In [ ]:
df_train.plot(kind='scatter', x='GDP per capita', y='Life satisfaction');

In [ ]:
df_train.plot(kind='scatter', x='Employment rate', y='Life satisfaction');

* まあ大丈夫そう。

### 特徴量とターゲットの用意

* 特徴量である(一人当たりのGDP, 雇用率)と、ターゲットである生活満足度とを、別々に保存する。

In [ ]:
X = df_train[['GDP per capita', 'Employment rate']]
y = df_train['Life satisfaction']

In [ ]:
X

In [ ]:
y

## k近傍法の実装（k=1の場合）

* 以下では、例題として、韓国の生活満足度の予測を考える。
  * そして、テストデータ上での生活満足度の予測をうまくいかせるために・・・
  * k近傍法をどのように使えばよいか、チューニングしていく。

* 言い換えれば、韓国のデータが、検証データとなる。
  * 検証データも、普通は、複数インスタンスを用意する。

### k近傍法の実装の仕方
* k=1に設定してk近傍法を使うとはどういうことか？
  1. 韓国に最も近い国を求める。
  2. 韓国に最も近い国の生活満足度を、韓国の生活満足度の予測値とする。

* 韓国に最も近い国を、どうやって選べばいいだろうか？

In [ ]:
X.loc[['Korea']]

* なお、以下のようにすれば、インデックス、つまり国名を枚挙できる。

In [ ]:
for country in X.index:
  print(country)

* 以下のようにすれば、各国の特徴量をNumpyの配列として枚挙できる。

In [ ]:
for country in X.index:
  print(X.loc[country].values)

* 以下、まずは簡単のため、雇用率はしばらく使わないことにし・・・
* 一人当たりのGDPだけを各国の特徴量として、生活満足度の予測を行うコードを書くことにする。

### 演習問題1
* 一人当たりのGDPが、韓国に最も近い国を求めてみよう。

### 演習問題2
* 上で求めた国の生活満足度を、韓国の生活満足度の予測値としたとき、予測誤差はいくらか。

## k近傍法の実装 （kの値が任意の場合）

* kの値が2以上の場合のk近傍法を実装する。
* そして、kの値がいくらのときに最も予測性能が良くなるかを調べることで・・・
* kの値をチューニングする。

### 演習問題3
* 全ての国について、一人当たりのGDPを使って計算される韓国との距離を求めよう。

### 演習問題4
* 韓国の一人当たりのGDPに近い順に、他の全ての国を並べ替えて表示させよう。

### 演習問題5
* 変数kに近傍の個数が代入されている。韓国のk個の近傍の国名を、韓国に近い順に表示させよう。

In [ ]:
k = 5
# ここに続きを書く

### 演習問題6
* 上で求めたk個の国について、それらの国の生活満足度の*平均値*を求めよう。

### 演習問題7
* 上の平均値を、韓国の生活満足度の予測値とした場合、予測誤差はいくらか。

# 課題

## 問1
* 変数kの値を、1から27まで変化させたとき、最も予測誤差が小さくなるのは、kの値がいくらのときか。

## 問2
* 一人当たりのGDPを使うのではなく、雇用率を使って生活満足度の予測を行ったとき、最も予測誤差を小さくするkの値はいくらか。